In [88]:
from typing import Literal
from collections import deque

In [89]:
class Node:
    """
    数式の要素を表すノード

    params:
    ----------
    value: str
        数式の 1 要素

    left: Node | None
        演算子の左辺

    right: Node | None
        演算子の右辺
    """

    def __init__(self, value: str, left=None, right=None) -> None:
        self.value: str = value
        self.left: Node | None = left
        self.right: Node | None = right

    def __str__(self) -> str:
        return self.value

In [90]:
def to_node(s: Node | str) -> Node:
    """
    数式の各要素をノードにして返す関数

    params:
    ----------
    s: Node | str
        数式の要素

    return:
    ----------
    Node
        数式の要素を表すノード
    """
    if isinstance(s, Node):
        return s
    return Node(s)


def post(formula: str) -> Node:
    """
    後置記法の数式を数式の木にして返す関数

    params:
    ----------
    formula: str
        数式

    return:
    ----------
    Node
        数式の木の根のノード
    """
    formula_deq = deque(formula.split())
    stack = []
    while len(formula_deq) > 0:
        s = formula_deq.popleft()
        if s in ["+", "-", "*", "/"]:
            right = stack.pop()
            left = stack.pop()
            new_node = Node(s, to_node(left), to_node(right))
            stack.append(new_node)
        else:
            stack.append(s)
    return stack[0]


def pre(formula: str) -> Node:
    """
    前置記法の数式を数式の木にして返す関数

    params:
    ----------
    formula: str
        数式

    return:
    ----------
    Node
        数式の木の根のノード
    """
    formula_list = formula.split()
    formula_list.reverse()
    stack = []
    formula_deq = deque(formula_list)
    while len(formula_deq) > 0:
        s = formula_deq.popleft()
        if s in ["+", "-", "*", "/"]:
            left = stack.pop()
            right = stack.pop()
            new_node = Node(s, to_node(left), to_node(right))
            stack.append(new_node)
        else:
            stack.append(s)
    return stack[0]


def infix(formula: str) -> Node:
    """
    中置記法の数式を数式の木にして返す関数

    params:
    ----------
    formula: str
        数式

    return:
    ----------
    Node
        数式の木の根のノード
    """
    order = {"+": 2, "-": 2, "*": 1, "/": 1, "(": 3}
    operator = []
    stack = []
    for s in formula.replace("(", " ( ").replace(")", " ) ").split():
        if s == ")":
            while True:
                t = operator.pop()
                if t == "(":
                    break
                stack.append(t)
        elif s == "(":
            operator.append(s)
        elif s in ["+", "-", "*", "/"]:
            while len(operator) > 0:
                t = operator.pop()
                if order[t] > order[s]:
                    operator.append(t)
                    break
                stack.append(t)
                if order[t] == order[s]:
                    break
            operator.append(s)
        else:
            stack.append(s)
    while len(operator) > 0:
        stack.append(operator.pop())
    return post(" ".join(stack))


def make_formula_tree(formula: str, type: Literal["pre", "in", "post"]) -> Node:
    """
    各記法の数式をそれぞれに合った、数式の木にして返す関数を呼び出す関数

    params:
    ----------
    formula: str
        数式
    type: Literal["pre", "in", "post"]
        数式の記法
        pre->前置記法
        in->中置記法
        post->後置記法

    return:
    ----------
    Node
        数式の木の根のノード
    """
    if type == "post":
        return post(formula)
    if type == "pre":
        return pre(formula)
    return infix(formula)

In [91]:
def return_preorder(node: Node):
    """
    数式の木を前置記法の数式にして返す関数

    params:
    ----------
    node: Node
        数式の木の根のノード

    return:
    ----------
    str
        数式
    """
    right = ""
    left = ""
    if node.left is not None:
        left = return_preorder(node.left)
    if node.right is not None:
        right = return_preorder(node.right)
    return f"{node.value} {left} {right}"


def return_inorder(node: Node):
    """
    数式の木を中置記法の数式にして返す関数

    params:
    ----------
    node: Node
        数式の木の根のノード

    return:
    ----------
    str
        数式
    """
    right = ""
    left = ""
    if node.value in ["*", "/"]:
        if node.left is not None:
            if node.left.value in ["+", "-"]:
                left = f"( {return_inorder(node.left)} ) "
            else:
                left = return_inorder(node.left)
        if node.right is not None:
            if node.right.value in ["+", "-"]:
                right = f"( {return_inorder(node.right)} ) "
            else:
                right = return_inorder(node.right)
    else:
        if node.left is not None:
            left = return_inorder(node.left)
        if node.right is not None:
            right = return_inorder(node.right)
    return f"{left} {node.value} {right}"


def return_postorder(node: Node):
    """
    数式の木を後置記法の数式にして返す関数

    params:
    ----------
    node: Node
        数式の木の根のノード

    return:
    ----------
    str
        数式
    """
    right = ""
    left = ""
    if node.left is not None:
        left = return_postorder(node.left)
    if node.right is not None:
        right = return_postorder(node.right)
    return f"{left} {right} {node.value}"

In [92]:
def change_formula_type(
    formula: str, expected_type: Literal["pre", "in", "post"]
) -> str:
    """
    各記法の数式を任意の記法の数式に変換する関数

    params:
    ----------
    formula: str
        数式
    expected_type: Literal["pre", "in", "post"]
        変換後の数式の記法
        pre->前置記法
        in->中置記法
        post->後置記法

    return:
    ----------
    Node
        変換後の数式
    """
    type: Literal["pre", "in", "post"]
    if formula[0] in ["+", "-", "*", "/"]:
        type = "pre"
    elif formula[len(formula) - 1] in ["+", "-", "*", "/"]:
        type = "post"
    else:
        type = "in"
    formula_tree = make_formula_tree(formula, type)
    result: str = {
        "pre": return_preorder,
        "in": return_inorder,
        "post": return_postorder,
    }[expected_type](formula_tree)
    return " ".join(result.split())